In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
# Additional libraries for data handling, preprocessing, etc.
import cv2
import os
import numpy as np

In [2]:
# Define input shape based on your image size (here, 640x360)
input_shape = (360, 640, 3)

# Load MobileNet, excluding the final classification layer
mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers to avoid retraining them (optional)
for layer in mobilenet.layers:
  layer.trainable = False


NameError: name 'MobileNet' is not defined

In [3]:
# GlobalAveragePooling2D reduces spatial dimensions to a single vector
x = GlobalAveragePooling2D()(mobilenet.output)

# Dense layer with 4 units (one for each variant) and softmax activation
predictions = Dense(2, activation='sigmoid')(x)

# Create the final model
model = tf.keras.Model(inputs=mobilenet.input, outputs=predictions)


In [4]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [7]:
import cv2
import os
import numpy as np

# Define empty lists for images and labels
X = []
y = []

# Folder path (replace 'path/to/images' with your actual folder path)
folder_path = 'classification_data/'

# Loop through folders
for folder_name in ['attack_img', 'explore_img']: #, 'loot_img', 'getaway_img']:
    folder_full_path = os.path.join(folder_path + folder_name)
    
    class_label = {
      'attack_img': [1,0],
      'explore_img': [0,1],
      # 'loot_img': [0,0,1,0],
      # 'getaway_img': [0,0,0,1]
    }[folder_name]
    
    for filename in os.listdir(folder_full_path):
        if filename.endswith(".jpg") :
            img = cv2.imread(os.path.join(folder_full_path, filename))
        
            # Append image to X and label to y
            X.append(img)
            
            
            y.append(class_label)
        

X = np.asarray(X)
y = np.asarray(y)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
# (print(y_train[-100:-1]))

(1330, 360, 640, 3) (1330, 2)


In [12]:
model.fit(X_train, y_train, epochs=80, validation_data=(X_val, y_val))


Epoch 1/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.4643 - loss: 1.3438 - val_accuracy: 0.7297 - val_loss: 0.7436
Epoch 2/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - accuracy: 0.6708 - loss: 0.7405 - val_accuracy: 0.7207 - val_loss: 0.5860
Epoch 3/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.7327 - loss: 0.5906 - val_accuracy: 0.7477 - val_loss: 0.5305
Epoch 4/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.7245 - loss: 0.5477 - val_accuracy: 0.7477 - val_loss: 0.5073
Epoch 5/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7091 - loss: 0.5116 - val_accuracy: 0.7297 - val_loss: 0.4857
Epoch 6/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.7392 - loss: 0.4987 - val_accuracy: 0.7387 - val_loss: 0.4728
Epoch 7/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7591 - loss: 0.4786 - val_accuracy: 0.7387 - val_loss: 0.4595
Epoch 8/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.7886 - loss: 0.4453 - val_accuracy: 0.7658 - val_loss:

In [13]:
loss, accuracy = model.evaluate(X_val, y_val)
print('Test accuracy:', accuracy)

model.save('classificator_2.h5')

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.8008 - loss: 0.4113


Test accuracy: 0.8198198080062866


In [10]:
model_rap = tf.keras.models.load_model('classificator_5.h5')


for n in range(20):
    y_real = y_val[n]
    tbp = np.asarray(X_val[n])
    cv2.imshow('ss', tbp)
    tbp = np.expand_dims(tbp, axis=0)
    predictions = model_rap.predict(tbp)
    print(predictions, "\n", y_real)

    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 922ms/step
[[0.8505496  0.08694281]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
[[0.6967798 0.1505777]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
[[0.9571435  0.03062754]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
[[0.19306839 0.4927955 ]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
[[0.49941036 0.20388314]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
[[0.9443833  0.02995677]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
[[0.27063748 0.47569102]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
[[0.32349434 0.3761934 ]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
[[0.2262347  0.59166986]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
[[0.8025678  0.15414624]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
[[0.8153183 0.1202962]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
[[0.93353856 0.03972495]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
[[0.84322953 0.05783981]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
[[0.93655133 0.05109823]] 
 [

In [ ]:
model_rap = tf.keras.models.load_model('classificator_4.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_rap.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
model_rap.save('classificator_5.h5')

augmented data fragment

In [2]:
X = []
y = []

folder_path = 'augmented_data/'

for folder_name in ['attack_img', 'explore_img']: #, 'loot_img', 'getaway_img']:
    folder_full_path = os.path.join(folder_path + folder_name)
    
    class_label = {
      'attack_img': [1,0],
      'explore_img': [0,1],
      # 'loot_img': [0,0,1,0],
      # 'getaway_img': [0,0,0,1]
    }[folder_name]
    
    for filename in os.listdir(folder_full_path):
        if filename.endswith(".jpg") :
            img = cv2.imread(os.path.join(folder_full_path, filename))
        
            # Append image to X and label to y
            X.append(img)
            
            
            y.append(class_label)
        

X = np.asarray(X)
y = np.asarray(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)

(539, 360, 640, 3) (539, 2)


In [ ]:
model_rap = tf.keras.models.load_model('classificator_5.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_rap.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
model_rap.save('classificator_5_augmented.h5')

Epoch 1/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.5478 - loss: 0.9194 - val_accuracy: 0.5333 - val_loss: 0.8030
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.6171 - loss: 0.7377 - val_accuracy: 0.6000 - val_loss: 0.7365
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6238 - loss: 0.6760 - val_accuracy: 0.6296 - val_loss: 0.7277
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6253 - loss: 0.7166 - val_accuracy: 0.6222 - val_loss: 0.7338
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6806 - loss: 0.6334 - val_accuracy: 0.6296 - val_loss: 0.7040
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6842 - loss: 0.6315 - val_accuracy: 0.6222 - val_loss: 0.7046
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6995 - loss: 0.6055 - val_accuracy: 0.5481 - val_loss: 0.7626
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.6577 - loss: 0.6432 - val_accuracy: 0.6519 - val_loss: